# ANÁLISIS EXPLORATORIO DE DATOS

## 1. IMPORTAMOS PAQUETES Y DATOS

In [190]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
%config IPCompleter.greedy = True
import warnings
warnings.filterwarnings('ignore')

Ruta archivo de calidad de datos

In [191]:
ruta_principal = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/EVOLVE/Data Science/EVOLVE/Fernando_Costa/Practicas/Mini_Proyecto_EDA/'
carpeta = '002_archivos/'

Nombres de los ficheros de datos.

In [192]:
df_calidad = 'trabajo_resultado_calidad.pickle'

Cargamos los datasets

In [193]:
df = pd.read_pickle(ruta_principal + carpeta + df_calidad)

## 2. CONCLUSIONES Y CAMBIOS NECESARIOS

Observamos lo siguiente sobre las variables categóricas y numéricas: La norma es dejar lo nulos como NaN 
- En ambos casos hay variables con muchos nulos pero tiene sentido que sean nulos porque son datos que no era necesario rellenarlos
    - crm_cd_4, crm_cd_3 y crm_cd_2 son los códigos de los crímenes secundarios. Entiendo que no hubo crímenes secundarios
    - weapon_used_cd y weapon_desc: Habla de las armas usadas. Entiendo que no hubo armas
    - cross_street: Identifica si el crimen se cometió en un cruce. Entiendo que no fue en un cruce
    - mocodes: Habla del modus operandi específico del crimen. Entiendo que no encontraron un patrón identificable

- En el caso de vict_descent vamos a borrar los 2 registros que hay con '-'
- En el caso de vict_sex, aparecen registros con ['M', 'F', nan, 'X', 'H', '-']. Vamos a dejar solamente M: Male, F: Female y U: Unknown y borraremos el registro con '-'
- premis_cd y premis_desc contienen información sobre las instalaciones donde se produjo el crimen (código y descripción). Desconocemos el significado de los códigos, pero observamos que los registros de (df.premis_cd.notna())&(df.premis_desc.isna()) se concentran en unos cuantos códigos y habría que investigarlos con una persona de dentro de la organización para obtener mas información. De momento:
    - premis_cd podrían ser que no lo hayan escrito o que no tengan código para ese sitio concreto y habría que investigarlo con alguien de dentro
    - premis_desc que son nulos son los mismos que premis_cd por lo que los dejamos como nulos
- En el caso de status, los vamos a borrar porque no hay información en casi ninguna variable y solamente es un caso 
- Por último, crm_cd_1. Este es el código del delito principal que siempre debe ser un código mas bajo (mas bajo = mas grave) que los códigos de crm_cd_1, 2 3 y 4. Así que vamos a ponerles el mismo código por practicar entendiendo que se equivocaron al rellenar el campo y pusieron la información en el crm_cd_2 en vez de el crm_cd_1
- Eliminación de variables:
    - Por baja frecuencia (<2%) ya que no podemos agrupar los datos para sacer estadísticos ni conclusiones de grupo
        - location
        - cross_street
        - time_occ
    - Hemos generado una columna alternativa llamada 'time_to_report' con la información y no aportan: time_occ
    Sobre 'time_to_report' si nos metieramos a hacer un análisis mas profundo, habría que segmentar esta variable en tramos para estudiar todos los casos que superan el percentil 75% que son muchos pero, para el caso de estudio y el objetivo del ejercicio, entiendo que no vamos a entrar en análisis tan profundo

In [194]:
df.isna().sum().sort_values(ascending=False)

crm_cd_4          331245
crm_cd_3          330494
crm_cd_2          308295
cross_street      280530
weapon_used_cd    223236
weapon_desc       223236
mocodes            49901
vict_descent       47573
vict_sex           47570
premis_desc          208
premis_cd              9
crm_cd_1               2
status                 1
date_rptd              0
date_occ               0
time_to_report         0
crm_cd_desc            0
part_1_2               0
crm_cd                 0
vict_age               0
time_occ               0
area_name              0
area                   0
rpt_dist_no            0
status_desc            0
location               0
lat                    0
lon                    0
year_rptd              0
month_rptd             0
day_rptd               0
year_occ               0
month_occ              0
day_occ                0
hour_occ               0
dtype: int64

### VARIABLES

#### MOCODES

Imputamos los nulos de 'mocodes' por 9999 para tenerlos identificados pero que sea un valor numérico de 4 dígitos

In [195]:
var_imputar_numero = ['mocodes']
numero = '9999'
df[var_imputar_numero] = df[var_imputar_numero].fillna(numero)
df.isna().sum().sort_values(ascending=False)

crm_cd_4          331245
crm_cd_3          330494
crm_cd_2          308295
cross_street      280530
weapon_used_cd    223236
weapon_desc       223236
vict_descent       47573
vict_sex           47570
premis_desc          208
premis_cd              9
crm_cd_1               2
status                 1
time_to_report         0
date_rptd              0
date_occ               0
mocodes                0
crm_cd_desc            0
crm_cd                 0
time_occ               0
vict_age               0
area                   0
area_name              0
rpt_dist_no            0
part_1_2               0
status_desc            0
location               0
lat                    0
lon                    0
year_rptd              0
month_rptd             0
day_rptd               0
year_occ               0
month_occ              0
day_occ                0
hour_occ               0
dtype: int64

#### STATUS

Borramos el registro de status

In [196]:
df.loc[(df.status.isna())]

,date_rptd,date_occ,time_to_report,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,...,cross_street,lat,lon,year_rptd,month_rptd,day_rptd,year_occ,month_occ,day_occ,hour_occ
dr_no,,,,,,,,,,,,,,,,,,,,,
241810568,2024-06-08,2024-05-24,15,1900,18,Southeast,1871,1,510,VEHICLE - STOLEN,...,NaN,33.9092,-118.2827,2024,06,08,2024,05,24,19


In [197]:
df = df.loc[df.status.notna()]
df.isna().sum().sort_values(ascending=False)

crm_cd_4          331244
crm_cd_3          330493
crm_cd_2          308294
cross_street      280529
weapon_used_cd    223235
weapon_desc       223235
vict_descent       47572
vict_sex           47569
premis_desc          207
premis_cd              8
crm_cd_1               2
time_to_report         0
date_occ               0
date_rptd              0
crm_cd                 0
mocodes                0
crm_cd_desc            0
time_occ               0
area                   0
status                 0
vict_age               0
area_name              0
rpt_dist_no            0
part_1_2               0
status_desc            0
location               0
lat                    0
lon                    0
year_rptd              0
month_rptd             0
day_rptd               0
year_occ               0
month_occ              0
day_occ                0
hour_occ               0
dtype: int64

#### VICT_DESCENT

Borramos los dos registros con '-' e imputamos los registros de vict_descent según la distribución de los datos 

In [198]:
df = df.loc[~(df.vict_descent=='-')]

#### CRM_CD_1

Imputamos los valores de crm_cd_1 que son nulos por los de crm_cd_2

In [199]:
df['crm_cd_1'] = df['crm_cd_1'].fillna(df['crm_cd_2'])
df.isna().sum().sort_values(ascending=False)

crm_cd_4          331244
crm_cd_3          330493
crm_cd_2          308294
cross_street      280529
weapon_used_cd    223235
weapon_desc       223235
vict_descent       47572
vict_sex           47569
premis_desc          207
premis_cd              8
date_rptd              0
time_to_report         0
date_occ               0
crm_cd                 0
part_1_2               0
mocodes                0
crm_cd_desc            0
time_occ               0
area                   0
status                 0
vict_age               0
area_name              0
rpt_dist_no            0
status_desc            0
crm_cd_1               0
location               0
lat                    0
lon                    0
year_rptd              0
month_rptd             0
day_rptd               0
year_occ               0
month_occ              0
day_occ                0
hour_occ               0
dtype: int64

In [200]:
#Cambiar formatos manualmente
df = df.astype({'crm_cd_1':'float64',
                'crm_cd_2':'float64'})

# Convertir columnas object a tipo 'category'
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = df[col].astype("category")

In [201]:
df.crm_cd_1.info()

<class 'pandas.core.series.Series'>
Index: 331263 entries, 200114944 to 240710787
Series name: crm_cd_1
Non-Null Count   Dtype   
--------------   -----   
331263 non-null  category
dtypes: category(1)
memory usage: 3.2 MB


#### VICT_SEX

Usaremos el SimpleImputer para aplicar sobre la variable 'vict_sex' que vamos a sustituir los nulos por 'U' y sobre 'premis_cd' imputaremos por la moda

In [202]:
#['M', 'F', nan, 'X', 'H', '-']

#Cargamos e instanciamos SimpleImputer
from sklearn.impute import SimpleImputer

#Reenplaza todos los 'unknown' de df y los hace NaN de Numpy para que después los identifique bien.
#El valor a reemplazar hay que mirarlo en cada caso
df.vict_sex.replace(['nan'], np.nan, inplace = True)

#Instanciamos el nuevo imputer
#Variables CATEGÓRICAS
nulos_var = SimpleImputer(strategy='constant', fill_value='Unknwon') #Reenplazamos por un valor constante

#Entrenamos el imputer
#Es importante que si hacemos una sola variable la pongamos con doble corchete. Si no, nos dará un error de dimensiones
nulos_var.fit(df[['vict_sex']])

#Ejecutamos el imputer
df[['vict_sex']] = nulos_var.transform(df[['vict_sex']])

Eliminamos el registro de vict_sex = '-'

In [203]:
df = df.loc[~(df.vict_sex=='-')]

Estandarización de los valores de vict_sex para que solamente haya 'Male', 'Female' y 'Unknown'

In [204]:
df.vict_sex.unique()

array(['F', 'Unknwon', 'M', 'X', 'H'], dtype=object)

In [205]:
df.vict_sex.replace({'M':'Female',
                     'F':'Female',
                     'H':'Male',
                     'X':'Male'}, inplace=True)

In [206]:
df.vict_sex.unique()

array(['Female', 'Unknwon', 'Male'], dtype=object)

#### LOCATION

In [207]:
df = df.drop(columns='location')

#### CROSS_STREET

In [208]:
df = df.drop(columns='cross_street')

#### TIME_OCC

In [209]:
df = df.drop(columns='time_occ')

### VALORES ATÍPICOS

Localizamos los valores atípicos

In [210]:
for variable in df:
    print(variable + '\n')
    print(df[variable].value_counts(dropna = False))
    print('\n\n')

date_rptd

date_rptd
2022-12-05    321
2023-02-03    306
2022-11-02    304
2022-05-02    303
2022-06-02    302
             ... 
2025-03-14      1
2025-03-19      1
2025-03-13      1
2025-06-04      1
2025-02-23      1
Name: count, Length: 1868, dtype: int64



date_occ

date_occ
2022-12-02    392
2022-06-01    366
2022-09-01    362
2023-01-01    360
2021-01-01    354
             ... 
2025-02-10      1
2025-03-18      1
2025-02-24      1
2025-01-14      1
2025-05-29      1
Name: count, Length: 1853, dtype: int64



time_to_report

time_to_report
0      159128
1       73016
2       21309
3       12145
4        7919
        ...  
647         1
975         1
529         1
911         1
974         1
Name: count, Length: 1164, dtype: int64



area

area
1     22915
12    20400
14    19580
3     19062
6     17102
15    17094
20    16524
18    16353
13    16199
7     15960
2     15339
8     15111
11    14330
9     14098
10    13874
17    13805
5     13643
21    13506
19    13339
4     12156

#### AGRUPACION DE CATEGORÍAS RARAS

In [211]:
# Función de agrupacion de variables raras
def agrupar_cat_raras(variable, criterio = 0.02):
    #Calcula las frecuencias
    frecuencias = variable.value_counts(normalize=True)
    #Identifica las que están por debajo del criterio
    temp = [cada for cada in frecuencias.loc[frecuencias < criterio].index.values]
    #Las recodifica en 'OTROS'
    temp2 = np.where(variable.isin(temp),'OTROS',variable)
    #Devuelve el resultado
    return(temp2)

# Variables a agrupar
var_imputar_cat_raras = ['weapon_used_cd','premis_cd','premis_desc','weapon_used_cd','weapon_desc','crm_cd_1','crm_cd_2','crm_cd_3','crm_cd_4']

# Aplicación
for variable in var_imputar_cat_raras:
    df[variable] = agrupar_cat_raras(df[variable],criterio = 0.02)

Hacemos un describe sobre las variables numéricas a ver que vemos y observamos que, aunque hay varias variables numéricas, no podríamos hacer ningún estadísticos sobre ellas, exceptopsobre 'vict_age' donde vemos que la media de edad es de 29 años y la mediana está en 22 años pero observamos que el mínimo es -4 y máximo 120 años. Esto lo solucionaremos una media winsorizada (0,100)

In [212]:
df.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
time_to_report,331263.0,12.071732,68.215594,0.0,0.0,1.0,2.0,1791.0
vict_age,331263.0,28.915635,22.000650,-4.0,0.0,30.0,44.0,120.0


In [213]:
df.vict_age.value_counts().sort_index(ascending=True)

vict_age
-4          1
-3          2
-2          8
-1         37
 0      88752
        ...  
 96        29
 97        28
 98        27
 99       125
 120        1
Name: count, Length: 104, dtype: int64

#### WINSORIZACIÓN

VICT_AGE: Realizamos una imputacion de los valores de la edad tras realizar una media winsorizada manualmente (0, 100)

In [214]:
variable = 'vict_age'
df[variable].describe().loc[['min','max']]

min     -4.0
max    120.0
Name: vict_age, dtype: float64

In [215]:
minimo = 0
maximo = 100
df[variable] = df[variable].clip(minimo, maximo)
df[variable].describe()

count    331263.000000
mean         28.915765
std          22.000170
min           0.000000
25%           0.000000
50%          30.000000
75%          44.000000
max         100.000000
Name: vict_age, dtype: float64

In [216]:
num = df.select_dtypes(include='number')
num

,time_to_report,vict_age
dr_no,,
200114944,1,27
202007973,1,0
202114730,0,36
210704099,12,51
200511531,8,66
...,...,...
241209917,0,29
241814866,1,0
240113722,0,0


In [217]:
cat = df.select_dtypes(exclude='number')
cat.head()

,date_rptd,date_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,vict_sex,...,crm_cd_4,lat,lon,year_rptd,month_rptd,day_rptd,year_occ,month_occ,day_occ,hour_occ
dr_no,,,,,,,,,,,,,,,,,,,,,
200114944,2020-07-17,2020-07-16,1,Central,111,1,330,BURGLARY FROM VEHICLE,1307 1822 0329 0352 1609 0344,Female,...,nan,34.0658,-118.2366,2020,07,17,2020,07,16,14
202007973,2020-03-27,2020-03-26,20,Olympic,2002,1,510,VEHICLE - STOLEN,9999,Unknwon,...,nan,34.0803,-118.3125,2020,03,27,2020,03,26,14
202114730,2020-10-11,2020-10-11,21,Topanga,2136,2,624,BATTERY - SIMPLE ASSAULT,0400,Female,...,nan,34.2029,-118.5943,2020,10,11,2020,10,11,19
210704099,2021-01-04,2020-12-23,7,Wilshire,711,1,330,BURGLARY FROM VEHICLE,0344 1601,Female,...,nan,34.0781,-118.3732,2021,01,04,2020,12,23,18
200511531,2020-07-07,2020-06-29,5,Harbor,529,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),1300 0344,Female,...,nan,33.7672,-118.2414,2020,07,07,2020,06,29,20


## 4. GUARDADO DE RESULTADOS

In [218]:
ruta_principal = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/EVOLVE/Data Science/EVOLVE/Fernando_Costa/Practicas/Mini_Proyecto_EDA/'
carpeta = '002_archivos/'

In [219]:
ruta_trabajo = ruta_principal + carpeta + 'trabajo_resultado_eda.pickle'
ruta_cat = ruta_principal + carpeta + 'cat_resultado_eda.pickle'
ruta_num = ruta_principal + carpeta + 'num_resultado_eda.pickle'

Guardar los archivos

In [220]:
df.to_pickle(ruta_trabajo)
cat.to_pickle(ruta_cat)
num.to_pickle(ruta_num)